In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import copy
import pandas as pd
import time
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Loading in data

In [2]:
print("Loading")
path_Ben_linux_1 = "/home/rubin/Research/data/metadata_IMU_EMG_allgestures_allusers(1).pkl"

start_time = time.time()
data_df = pd.read_pickle(path_Ben_linux_1)
end_time = time.time()
print(f"Completed in {end_time - start_time}s")

Loading
Completed in 0.08310842514038086s


# Split Dataset into train and test split

In [3]:
# Impaired participants
pIDs_impaired = ['P102', 'P103', 'P104', 'P105', 'P106', 'P107', 'P108', 'P109', 'P110', 'P111',
                 'P112', 'P114', 'P115', 'P116', 'P118', 'P119', 'P121', 'P122', 'P123', 'P124', 
                 'P125', 'P126', 'P127', 'P128', 'P131', 'P132']

# Unimpaired participants
pIDs_unimpaired = ['P004', 'P005', 'P006', 'P008', 'P010', 'P011']

def split_and_preprocess_by_user(data_df, pIDs_impaired, pIDs_unimpaired, test_size=0.2):
    # Split impaired and unimpaired participants into train and test sets
    impaired_train, impaired_test = train_test_split(pIDs_impaired, test_size=test_size, random_state=42)
    unimpaired_train, unimpaired_test = train_test_split(pIDs_unimpaired, test_size=test_size, random_state=42)

    # Combine train and test users
    training_users = impaired_train + unimpaired_train
    test_users = impaired_test + unimpaired_test

    metadata_cols = ['Participant', 'Gesture_ID', 'Gesture_Num']

    # Split data by users
    train_df = data_df[data_df['Participant'].isin(training_users)]
    test_df = data_df[data_df['Participant'].isin(test_users)]

    # Subset metadata columns for training and testing sets
    train_metadata_df = train_df[metadata_cols].reset_index(drop=True)
    test_metadata_df = test_df[metadata_cols].reset_index(drop=True)

    # Drop metadata columns from the dataframes
    train_df = train_df.drop(metadata_cols, axis=1).reset_index(drop=True)
    test_df = test_df.drop(metadata_cols, axis=1).reset_index(drop=True)

    # Scale the data
    train_scaler = StandardScaler()

    # Fit on training data and transform both train and test sets
    ppd_train_df = pd.DataFrame(train_scaler.fit_transform(train_df))
    ppd_test_df = pd.DataFrame(train_scaler.transform(test_df))

    # Split IMU and EMG data
    ppd_train_imu_df = ppd_train_df.iloc[:, :72]
    ppd_train_emg_df = ppd_train_df.iloc[:, 72:]
    ppd_test_imu_df = ppd_test_df.iloc[:, :72]
    ppd_test_emg_df = ppd_test_df.iloc[:, 72:]

    # Concatenate metadata back to the processed data
    ppd_train_imu_df = pd.concat([train_metadata_df, ppd_train_imu_df], axis=1)
    ppd_train_emg_df = pd.concat([train_metadata_df, ppd_train_emg_df], axis=1)
    ppd_test_imu_df = pd.concat([test_metadata_df, ppd_test_imu_df], axis=1)
    ppd_test_emg_df = pd.concat([test_metadata_df, ppd_test_emg_df], axis=1)

    return ppd_train_imu_df, ppd_train_emg_df, ppd_test_imu_df, ppd_test_emg_df

# Call the function with the full participant list
ppd_train_imu_df, ppd_train_emg_df, ppd_test_imu_df, ppd_test_emg_df = split_and_preprocess_by_user(data_df, pIDs_impaired, pIDs_unimpaired)


In [4]:
print(ppd_train_imu_df.shape)
ppd_train_emg_df.head()

(153600, 75)


,Participant,Gesture_ID,Gesture_Num,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87
0,P103,move,1,-0.272553,-0.174909,-0.487032,-0.479092,-0.282206,-0.340962,-0.349967,-0.372957,-0.460476,-0.322696,-0.353601,-0.359954,-0.341444,-0.270736,-0.129773,-0.256569
1,P103,move,1,-0.237541,-0.219476,-0.491541,-0.456715,-0.259345,-0.269619,-0.370232,-0.354646,-0.456982,-0.301867,-0.349724,-0.356034,-0.337765,-0.281176,-0.106944,-0.250600
2,P103,move,1,-0.226202,-0.227727,-0.498172,-0.514807,-0.247238,-0.248624,-0.340924,-0.330598,-0.453981,-0.294390,-0.355787,-0.360192,-0.341552,-0.274673,-0.068181,-0.182230
3,P103,move,1,-0.059969,-0.167775,-0.476214,-0.597320,-0.266217,-0.281169,-0.308048,-0.280282,-0.518521,-0.294813,-0.355749,-0.350663,-0.333018,-0.264322,-0.065513,-0.144634
4,P103,move,1,-0.012488,-0.202698,-0.483415,-0.510942,-0.278117,-0.243418,-0.328787,-0.316063,-0.439284,-0.295585,-0.346027,-0.350198,-0.327595,-0.256635,-0.112046,-0.256429


In [5]:
import numpy as np
import torch
from torch.utils.data import Dataset

class EMG_IMU_Dataset(Dataset):
    def __init__(self, emg_df, imu_df, num_channels_emg=16, num_channels_imu=72, time_units=64):
        # Create labels array using only Gesture_ID from EMG data
        self.labels = emg_df['Gesture_ID'].values
        
        # Exclude metadata columns and reshape the EMG and IMU data
        emg_data = emg_df.drop(['Participant', 'Gesture_ID', 'Gesture_Num'], axis=1).values
        imu_data = imu_df.drop(['Participant', 'Gesture_ID', 'Gesture_Num'], axis=1).values

        # EMG data processing: (num_samples, time_units, num_channels_emg)
        num_samples_emg = len(emg_data) // time_units
        self.emg_data = emg_data.reshape(num_samples_emg, time_units, num_channels_emg).transpose((0, 2, 1))

        # IMU data processing: (num_samples, time_units, num_channels_imu)
        num_samples_imu = len(imu_data) // time_units
        self.imu_data = imu_data.reshape(num_samples_imu, time_units, num_channels_imu).transpose((0, 2, 1))

        # Create a dictionary to map each unique Gesture_ID to an integer label
        unique_labels = np.unique(self.labels)
        self.label_map = {label: i for i, label in enumerate(unique_labels)}
        
        # Map labels to integers
        self.labels = np.array([self.label_map[label] for label in self.labels[:num_samples_emg * time_units:time_units]])

        # Create a dictionary to map (Participant, Gesture_ID, Gesture_Num) to index
        self.index_map = {(row['Participant'], row['Gesture_ID'], row['Gesture_Num']): idx // time_units 
                          for idx, row in emg_df.iterrows()}

        # Sanity check
        print(f"EMG Data shape: {self.emg_data.shape}")
        print(f"IMU Data shape: {self.imu_data.shape}")
        print(f"Labels shape: {self.labels.shape}")
        print(f"Label mapping: {self.label_map}")
    
    def __len__(self):
        return len(self.emg_data)

    def __getitem__(self, idx):
        if isinstance(idx, tuple):
            # Get item by (Participant, Gesture_ID, Gesture_Num)
            idx = self.index_map[idx]
        
        emg_data = torch.tensor(self.emg_data[idx], dtype=torch.float32)
        imu_data = torch.tensor(self.imu_data[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        
        return emg_data, imu_data, label


In [6]:
from torch.utils.data import DataLoader

# Create dataset instances
train_dataset = EMG_IMU_Dataset(ppd_train_emg_df, ppd_train_imu_df)
test_dataset = EMG_IMU_Dataset(ppd_test_emg_df, ppd_test_imu_df)

# Create DataLoader instances
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


EMG Data shape: (2400, 16, 64)
IMU Data shape: (2400, 72, 64)
Labels shape: (2400,)
Label mapping: {'close': 0, 'delete': 1, 'duplicate': 2, 'move': 3, 'open': 4, 'pan': 5, 'rotate': 6, 'select-single': 7, 'zoom-in': 8, 'zoom-out': 9}
EMG Data shape: (800, 16, 64)
IMU Data shape: (800, 72, 64)
Labels shape: (800,)
Label mapping: {'close': 0, 'delete': 1, 'duplicate': 2, 'move': 3, 'open': 4, 'pan': 5, 'rotate': 6, 'select-single': 7, 'zoom-in': 8, 'zoom-out': 9}


In [50]:
import torch.nn as nn

class EMG_IMU_CNN(nn.Module):
    def __init__(self, num_classes=10, dropout_prob=0.5):
        super(EMG_IMU_CNN, self).__init__()

        # Convolutional Block for EMG Data (16 channels)
        self.emg_conv1 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.emg_bn1 = nn.BatchNorm1d(32)
        self.emg_conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.emg_bn2 = nn.BatchNorm1d(64)
        self.emg_conv3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.emg_bn3 = nn.BatchNorm1d(128)
        
        # Convolutional Block for IMU Data (72 channels)
        self.imu_conv1 = nn.Conv1d(in_channels=72, out_channels=32, kernel_size=3, padding=1)
        self.imu_bn1 = nn.BatchNorm1d(32)
        self.imu_conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.imu_bn2 = nn.BatchNorm1d(64)
        self.imu_conv3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.imu_bn3 = nn.BatchNorm1d(128)
        
        # Pooling layer (shared)
        self.pool = nn.MaxPool1d(kernel_size=2)

        # Dropout
        self.dropout = nn.Dropout(p=dropout_prob)

        # Fully connected layers after concatenation
        self.fc1 = nn.Linear(128 * 8 * 2, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, num_classes)

    def forward(self, emg_input, imu_input):
        # Forward pass for EMG data
        x_emg = torch.relu(self.emg_bn1(self.emg_conv1(emg_input)))
        x_emg = self.pool(x_emg)
        x_emg = torch.relu(self.emg_bn2(self.emg_conv2(x_emg)))
        x_emg = self.pool(x_emg)
        x_emg = torch.relu(self.emg_bn3(self.emg_conv3(x_emg)))
        x_emg = self.pool(x_emg)
        x_emg = self.dropout(x_emg)  # Dropout after final EMG conv block
        
        # Forward pass for IMU data
        x_imu = torch.relu(self.imu_bn1(self.imu_conv1(imu_input)))
        x_imu = self.pool(x_imu)
        x_imu = torch.relu(self.imu_bn2(self.imu_conv2(x_imu)))
        x_imu = self.pool(x_imu)
        x_imu = torch.relu(self.imu_bn3(self.imu_conv3(x_imu)))
        x_imu = self.pool(x_imu)
        x_imu = self.dropout(x_imu)  # Dropout after final IMU conv block

        # Flatten both EMG and IMU outputs
        x_emg = torch.flatten(x_emg, start_dim=1)
        x_imu = torch.flatten(x_imu, start_dim=1)

        # Concatenate the EMG and IMU outputs
        x = torch.cat((x_emg, x_imu), dim=1)

        # Pass through the fully connected layers with dropout
        x = self.dropout(torch.relu(self.fc1(x)))
        x = self.dropout(torch.relu(self.fc2(x)))
        x = self.fc3(x)
        
        return x


In [51]:
def train(model, optimizer, dataloader, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for emg_data, imu_data, labels in dataloader:
        # Move EMG data, IMU data, and labels to the device (GPU or CPU)
        emg_data, imu_data, labels = emg_data.to(device), imu_data.to(device), labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass with both EMG and IMU data
        outputs = model(emg_data, imu_data)
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()

        # Update the weights
        optimizer.step()

        # Update running loss
        running_loss += loss.item()
        
        # Compute accuracy
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    # Calculate final accuracy for the epoch
    accuracy = 100. * correct / total
    return running_loss / len(dataloader), accuracy


In [52]:
def evaluate(model, dataloader, criterion, device):
    model.eval()  # Set model to evaluation mode
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():  # Disable gradient computation
        for emg_data, imu_data, labels in dataloader:
            # Move EMG data, IMU data, and labels to the device (GPU or CPU)
            emg_data, imu_data, labels = emg_data.to(device), imu_data.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(emg_data, imu_data)
            loss = criterion(outputs, labels)
            
            # Update running loss
            running_loss += loss.item()
            
            # Compute accuracy
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    # Compute accuracy over the whole dataset
    accuracy = 100. * correct / total
    return running_loss / len(dataloader), accuracy


In [53]:
def train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs, device):   
    model.to(device)
    for epoch in range(num_epochs):
        start_time = time.time()

        # Train the model for one epoch
        train_loss, train_acc = train(model, optimizer, train_loader, criterion, device)

        end_time = time.time()
        epoch_time = end_time - start_time

        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        print(f"Epoch Time: {epoch_time:.2f} seconds")
        test_loss, test_acc = evaluate(model, test_loader, criterion, device)
        print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")


In [64]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize EMG_IMU_CNN model and move it to the device
emgimu_model = EMG_IMU_CNN().to(device)

# Number of epochs
num_epochs = 100

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(emgimu_model.parameters(), lr=0.0001, weight_decay=1e-3)

# Train the EMG-IMU model
train_model(emgimu_model, train_loader, test_loader, criterion, optimizer, num_epochs, device)



Epoch 1/100
Train Loss: 2.3173, Train Acc: 10.54%
Epoch Time: 0.54 seconds
Test Loss: 2.3408, Test Acc: 10.62%
Epoch 2/100
Train Loss: 2.2884, Train Acc: 12.21%
Epoch Time: 0.55 seconds
Test Loss: 2.3615, Test Acc: 11.50%
Epoch 3/100
Train Loss: 2.2789, Train Acc: 13.46%
Epoch Time: 0.51 seconds
Test Loss: 2.3823, Test Acc: 11.50%
Epoch 4/100
Train Loss: 2.2603, Train Acc: 15.46%
Epoch Time: 0.52 seconds
Test Loss: 2.3929, Test Acc: 11.62%
Epoch 5/100
Train Loss: 2.2426, Train Acc: 15.33%
Epoch Time: 0.52 seconds
Test Loss: 2.4574, Test Acc: 12.25%
Epoch 6/100
Train Loss: 2.2055, Train Acc: 18.79%
Epoch Time: 0.50 seconds
Test Loss: 2.5099, Test Acc: 10.88%
Epoch 7/100
Train Loss: 2.1665, Train Acc: 21.75%
Epoch Time: 0.52 seconds
Test Loss: 2.7117, Test Acc: 10.38%
Epoch 8/100
Train Loss: 2.1090, Train Acc: 24.21%
Epoch Time: 0.52 seconds
Test Loss: 3.1601, Test Acc: 11.50%
Epoch 9/100
Train Loss: 2.0494, Train Acc: 27.58%
Epoch Time: 0.52 seconds
Test Loss: 3.3304, Test Acc: 11.25%
E